In [1]:
import os
import numpy as np
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredPDFLoader,UnstructuredFileLoader,OnlinePDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
import faiss
import os

# Lire les documents texte depuis un dossier
def load_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='UTF-8') as file:
                documents.append(file.read())
    return documents

folder_path = "documents/"
documents = load_documents(folder_path)


In [3]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Définir le modèle d'encodage
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

# Encoder les documents
document_embeddings = embed_model.embed_documents(documents)


C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
import faiss
import numpy as np

document_embeddings = np.array(document_embeddings)
d = document_embeddings.shape[1]
#print(d)  
index = faiss.IndexFlatL2(d)
document_embeddings = document_embeddings.astype('float32')
index.add(document_embeddings)


In [5]:
# Encoder une requête
query = "C'est qui le coordinnateur de la MST: intelligence artificielle et sciences des données  ?"
query_embedding = embed_model.embed_query(query)

# Convertir la requête en numpy array et s'assurer que c'est un tableau 2D
query_embedding = np.array(query_embedding).astype('float32').reshape(1, -1)

# Effectuer une recherche
k = 3
distances, indices = index.search(query_embedding, k)

# Afficher les résultats
for i in range(k):
    print(f"Document {indices[0][i]} avec distance {distances[0][i]}")
    print(documents[indices[0][i]])
    print()

Document 298 avec distance 0.6787811517715454
Filière: MST : Intelligence Artificielle et Sciences de Données

Objectif: Cette formation d'excellence offre de solides connaissances en conception de systèmes d'intelligence artificielle et mathématiques appliquées afin de couvrir l'ensemble des problématiques de traitement et d'analyse des données massives que rencontre les entreprises. Elle met l'accent sur l'articulation entre apprentissage automatique, gestion et fouille de grandes masses de données, paradigmes du Big Data, représentation des connaissances, le traitement des données et sur les méthodologies récemment développées.

Coordinateur: Pr.EZZIYYANI MOSTAFA
Email: mezziyyani@uae.ac.ma

Débouchés: 

Programme:
Semestre 1:
  1. THÉORIES ET SYSTÈMES DE RAISONNEMENTS INTELLIGENTS
  2. MATHÉMATIQUES POUR ANALYSE DE DONNÉES
  3. PROGRAMMATION AVANCÉE
  4. BASES DE DONNÉES AVANCÉES
  5. MACHINE LEARNING 1
  6. ANGLAIS ET TECHNIQUE D’EXPRESSION

Semestre 2:
  1. INFRASTRUCTURE ET AR

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
class FAISSRetriever:
    def __init__(self, index, documents):
        self.index = index
        self.documents = documents
    
    def retrieve(self, query_embedding, k=3):
        query_embedding = np.array(query_embedding).astype('float32').reshape(1, -1)
        distances, indices = self.index.search(query_embedding, k)
        return [(self.documents[i], distances[0][j]) for j, i in enumerate(indices[0])]

# Charger un modèle de génération (par exemple, un modèle de la famille T5)
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model_ = AutoModelForSeq2SeqLM.from_pretrained("t5-base")


In [7]:
# Encoder une requête
query = "C'est qui le coordinnateur de la MST: intelligence artificielle et sciences des données  ?"
query_embedding = embed_model.embed_query(query)

# Utiliser le retrieveur pour obtenir les documents les plus pertinents
retriever = FAISSRetriever(index, documents)
results = retriever.retrieve(query_embedding)

# Préparer la requête et les documents contextuels pour la génération
input_text = query + " " + " ".join([doc for doc, dist in results])
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=2000, truncation=True)

# Générer la réponse
outputs = model_.generate(inputs, max_length=1000, num_beams=2, early_stopping=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Réponse générée:", response)


Réponse générée: Cette formation d'excellence offre de solides connaissances en conception de systèmes d'intelligence artificielle et mathématiques appliquées afin de couvrir l'ensemble des problématiques de traitement et d'analyse des données que rencontrent les entreprises.


In [8]:
"""from torch import cuda, bfloat16
import transformers
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, disk_offload

model_id = "NousResearch/Llama-2-7b-chat-hf"

# Determine the device to use
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Set quantization configuration to load a large model with less GPU memory
# Requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# HF authentication token
hf_auth = 'HF_AUTH_TOKEN'

# Initialize model configuration
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

# Load the model with weights
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    token=hf_auth,
)

# Define the device map
device_map = {
    'transformer.wte': 'cpu',  
    'transformer.h': 'cpu',    
    'lm_head': 'cpu'           
}

# Specify a directory for offloading
offload_dir = 'C:/Users/drief/Desktop/NLP PROJECT/llama2'

# Ensure the offload directory exists
import os
if not os.path.exists(offload_dir):
    os.makedirs(offload_dir)

# Use disk_offload to handle large model offloading
disk_offload(
    model=model,
    offload_dir=offload_dir
)

model.eval()
print(f"Model loaded and offloaded successfully")
"""

'from torch import cuda, bfloat16\nimport transformers\nfrom accelerate import init_empty_weights, load_checkpoint_and_dispatch, disk_offload\n\nmodel_id = "NousResearch/Llama-2-7b-chat-hf"\n\n# Determine the device to use\ndevice = f\'cuda:{cuda.current_device()}\' if cuda.is_available() else \'cpu\'\n\n# Set quantization configuration to load a large model with less GPU memory\n# Requires the `bitsandbytes` library\nbnb_config = transformers.BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type=\'nf4\',\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_compute_dtype=bfloat16\n)\n\n# HF authentication token\nhf_auth = \'HF_AUTH_TOKEN\'\n\n# Initialize model configuration\nmodel_config = transformers.AutoConfig.from_pretrained(\n    model_id,\n    use_auth_token=hf_auth\n)\n\n# Load the model with weights\nmodel = transformers.AutoModelForCausalLM.from_pretrained(\n    model_id,\n    trust_remote_code=True,\n    config=model_config,\n    quantization_config=bnb_config,\